In [ ]:
!pip install torch transformers gradio ai71 openai sentence-transformers gradio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
import numpy as np
import io
import matplotlib.pyplot as plt
import pandas as pd
import gradio as gr
import os
import random
from PIL import Image

from sentence_transformers import SentenceTransformer
from transformers import pipeline
from datetime import datetime
from PIL import Image
from datetime import datetime
from openai import OpenAI
from ai71 import AI71

In [ ]:
if torch.cuda.is_available():
    model = model.to('cuda')

NameError: name 'model' is not defined

In [ ]:
embed_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
classifier = pipeline("zero-shot-classification", model ='facebook/bart-large-mnli')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
with open ('emotion_group_labels.txt') as file:
    emotion_group_labels = file.read().splitlines()

In [ ]:
dials_embeddings = pd.read_pickle('kaggle_therapy_embeddings.pkl')
dials_embeddings.shape

(71894, 4)

In [ ]:
session_conversation=[]
AI71_BASE_URL = "https://api.ai71.ai/v1/"
AI71_API_KEY = os.getenv('AI71_API_KEY')

In [ ]:
# Measure cosine similarity between a pair of vectors
def cosine_distance(vec1,vec2):
  cosine = (np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))
  return cosine

In [ ]:
# Detect emotions from patient dialogues
def detect_emotions(text):
  emotion = classifier(text, candidate_labels=emotion_group_labels, batch_size=16)
  top_5_scores = [i/sum(emotion['scores'][:5]) for i in emotion['scores'][:5]]
  top_5_emotions = emotion['labels'][:5]
  emotion_set = {l: "{:.2%}".format(s) for l, s in zip(top_5_emotions, top_5_scores)}
  return emotion_set

In [ ]:
# Generate an image of trigger emotions
def generate_triggers_img(items):
    labels = list(items.keys())
    values = [float(v.strip('%')) for v in items.values()]  # Convert to float for plotting

    new_items = {k:v for k, v in zip(labels, values)}
    new_items = dict(sorted(new_items.items(), key=lambda item: item[1]))
    labels = list(new_items.keys())
    values = list(new_items.values())

    fig, ax = plt.subplots(figsize=(10, 6))
    colors = plt.cm.viridis(np.linspace(0, 1, len(labels)))

    bars = ax.barh(labels, values, color=colors)

    for spine in ax.spines.values():
        spine.set_visible(False)

    ax.tick_params(axis='y', labelsize=18)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_ticks_position('none')

    for bar in bars:
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height()/2, f'{width:.2f}%',
                ha='left', va='center', fontweight='bold', fontsize=18)

    plt.tight_layout()
    plt.savefig('triggeres.png')
    triggers_img = Image.open('triggeres.png')
    return triggers_img

In [ ]:
# Generate therapist responses and patient triggers
def get_doc_response_emotions(user_message, therapy_session_conversation):
    global session_conversation

    user_messages = []
    user_messages.append(user_message)
    emotion_set = detect_emotions(user_message)
    print(emotion_set)

    emotions_msg = generate_triggers_img(emotion_set)
    user_embedding = embed_model.encode(user_message, device='cuda' if torch.cuda.is_available() else 'cpu')

    similarities =[]
    for v in dials_embeddings['embeddings']:
      similarities.append(cosine_distance(user_embedding,v))

    top_match_index = similarities.index(max(similarities))
    # doc_response = dials_embeddings.iloc[top_match_index+1]['Doctor']
    doc_response = dials_embeddings.iloc[top_match_index]['Doctor']

    therapy_session_conversation.append(["User: "+user_message, "Therapist: "+doc_response])

    session_conversation.extend(["User: "+user_message, "Therapist: "+doc_response])

    print(f"User's message: {user_message}")
    print(f"RAG Matching message: {dials_embeddings.iloc[top_match_index]['Patient']}")
    # print(f"Therapist's response: {dials_embeddings.iloc[top_match_index+1]['Doctor']}\n\n")
    print(f"Therapist's response: {dials_embeddings.iloc[top_match_index]['Doctor']}\n\n")

    return '', therapy_session_conversation, emotions_msg

In [ ]:
def get_doc_response_emotions(user_message, therapy_session_conversation):
    global session_conversation

    user_messages = []
    user_messages.append(user_message)
    emotion_set = detect_emotions(user_message)
    print(emotion_set)

    emotions_msg = generate_triggers_img(emotion_set)
    user_embedding = embed_model.encode(user_message, device='cuda' if torch.cuda.is_available() else 'cpu')

    similarities =[]
    for v in dials_embeddings['embeddings']:
      similarities.append(cosine_distance(user_embedding,v))

    top_match_index = similarities.index(max(similarities))
    doc_response = dials_embeddings.iloc[top_match_index+1]['Doctor']
    therapy_session_conversation.append(["User: "+user_message, "Therapist: "+doc_response])

    session_conversation.extend(["User: "+user_message, "Therapist: "+doc_response])

    print(f"User's message: {user_message}")
    print(f"RAG Matching message: {dials_embeddings.iloc[top_match_index]['Patient']}")
    print(f"Therapist's response: {dials_embeddings.iloc[top_match_index+1]['Doctor']}\n\n")
    return '', therapy_session_conversation, emotions_msg

In [ ]:
# Generate summarization and recommendations for teh session
def summarize_and_recommend():
  global session_conversation
  session_time = str(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
  session_conversation_processed = session_conversation.copy()
  session_conversation_processed.insert(0, "Session_time: "+session_time)
  session_conversation_processed ='\n'.join(session_conversation_processed)
  print("Session conversation:", session_conversation_processed)

  AI71_BASE_URL = "https://api.ai71.ai/v1/"

  client = OpenAI(
      api_key=AI71_API_KEY,
      base_url=AI71_BASE_URL,
  )

  full_summary = ""
  for chunk in AI71(AI71_API_KEY).chat.completions.create(
      model="tiiuae/falcon-180b-chat",
      messages=[
          {"role": "system", "content": """You are an Expert Cognitive Behavioural Therapist and Precis writer.
          Summarize the below user content <<<session_conversation_processed>>> into useful, ethical, relevant and realistic phrases with a format
        Session Time:
        Summary of the patient messages: #in two to four sentences
        Summary of therapist messages: #in two to three sentences:
        Summary of the whole session: # in two to three sentences. Ensure the entire session summary strictly does not exceed 100 tokens."""},
          {"role": "user", "content": session_conversation_processed},
      ],
      stream=True,
  ):
      if chunk.choices[0].delta.content:
          summary = chunk.choices[0].delta.content
          # print("Chunk summary:", summary, sep="", end="", flush=True)
          full_summary += summary
  full_summary = full_summary.replace('User:', '').strip()
  print("\n")
  print("Full summary:", full_summary)

  full_recommendations = ""
  for chunk in AI71(AI71_API_KEY).chat.completions.create(
      model="tiiuae/falcon-180b-chat",
      messages=[
          {"role": "system", "content": """You are an expert Cognitive Behavioural Therapist.
          Based on the full summary <<<full_summary>>> provide clinically valid, useful, appropriate action plan for the Patient as a bullted list.
          The list shall contain both medical and non medical prescriptions, dos and donts. The format of response shall be in passive voice with proper tense.
        - The patient is referred to........ #in one sentence
        - The patient is advised to ........ #in one sentence
        - The patient is refrained from........ #in one sentence
        - It is suggested  that tha patient ........ #in one sentence
        - Scheduled a follow-up session with the patient........#in one sentence
          *Ensure the list contains NOT MORE THAN 7 points"""},
          {"role": "user", "content": full_summary},
      ],
      stream=True,
  ):
      if chunk.choices[0].delta.content:
          rec = chunk.choices[0].delta.content
          # print("Chunk recommendation:", rec, sep="", end="", flush=True)
          full_recommendations += rec
  full_recommendations = full_recommendations.replace('User:', '').strip()
  print("\n")
  print("Full recommendations:", full_recommendations)

  return full_summary, full_recommendations

In [ ]:
img_paths = ['2.jpg','3.jpeg','4.jpeg','5.jpeg','6.jpeg']
img_path = random.choice(img_paths)
img_1 = Image.open(img_path)

img_path = '7.jpg'
img_2 = Image.open(img_path)

with gr.Blocks() as demo:
    gr.Markdown("""# Falcon Cognitive Behavioural Therapy Assistant
    - Your Personal AI Therapist. Start chatting...""")
    therapy_session_conversation = gr.State([])

    with gr.Row():

        with gr.Column(scale=1):
            image = gr.Image(type='pil', label ='Your Personal Therapy Assistant', value=img_1, interactive=False)
            emotions = gr.Image(value=img_2, label='Top 5 Emotion Triggers')
            summary_notes = gr.Textbox(label="Summary Notes of the Session", visible=False)

        with gr.Column(scale=2):
            chatbox = gr.Chatbot(label="Therapy Session Conversation",value =[[None, 'Therapist: Hello, What can I do for you?']], height=300)
            user_input = gr.Textbox(placeholder="Enter your message here...", label="User")
            submit_button = gr.Button("Submit")
            submit_button.click(get_doc_response_emotions, [user_input, chatbox], [user_input, chatbox, emotions])
            user_input.submit(get_doc_response_emotions, [user_input, chatbox], [user_input, chatbox, emotions])
            recommendations = gr.Textbox(label="Recommended Actions", visible = False)

            def summarize_and_recommend_process():
              sn, r = summarize_and_recommend()
              return gr.update(visible=True, value=sn), gr.update(visible=True, value=r)

    process_button = gr.Button("Generate Session Notes & Recommendations")
    clear = gr.ClearButton(components=[user_input, chatbox, emotions, summary_notes, recommendations], value="Clear console")
    process_button.click(summarize_and_recommend_process, inputs=None, outputs=[summary_notes, recommendations])


demo.launch(debug=True, share=True)


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:986: UserWarning: Expected 1 arguments for function <function summarize_and_recommend at 0x7bcbaae67880>, received 0.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:990: UserWarning: Expected at least 1 arguments for function <function summarize_and_recommend at 0x7bcbaae67880>, received 0.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8befc4f903cb35a401.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://8befc4f903cb35a401.gradio.live
